# Motor classification project :

This project consists of classification of hand motor movements during a non-visual vestibular deadreckoning task.

The cells are organized in terms of : 1. data standardization, 2. data preprocessing, 3. metric creation, 4. classification label creation (response classification, control classification), 5. classification, 6. correlation of perceptual and physial disorientation (SSQ).

In [1]:
%load_ext autoreload 
%autoreload 2

In [75]:
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import pickle

PCtype = 'Linux'

varr = {}
if PCtype == 'Windows':
    # Windows
    varr['main_path'] = "C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Time_series_analysis\\Motor_classification\\Motor_classification"  
    varr['main_path1'] = "%s\\a_data_standardization" % (varr['main_path'])
    varr['main_path2'] = "%s\\b_data_preprocessing" % (varr['main_path'])
    varr['main_path3'] = "%s\\c_calculate_metrics" % (varr['main_path'])
    filemarker = '\\'
elif PCtype == 'Linux':
    # Linux
    # varr['main_path'] = "/home/oem2/Documents/PROGRAMMING/9_Motor_classification_2018-22/Coding_version3_python_FINAL"
    varr['main_path'] = "/home/oem2/Documents/GITarea/Motor_classification"
    varr['main_path1'] = "%s/a_data_standardization" % (varr['main_path'])
    varr['main_path2'] = "%s/b_data_preprocessing" % (varr['main_path'])
    varr['main_path3'] = "%s/c_calculate_metrics" % (varr['main_path'])
    filemarker = '/'

import sys
sys.path.insert(1, '%s' % (varr['main_path']))

In [76]:
def load_dat_pickle(file_name="outSIG.pkl"):
    open_file = open(file_name, "rb")
    dataout = pickle.load(open_file)
    open_file.close()
    return dataout

# [A] Data standardization :
This is divided into two steps : 1. calculating the if the cabin moved after joystick movement and in what direction for all possible combinations of categories, 2. finding which combination of categores has the most "cabin-joystick follow". 

## [step 1] Calculating if the cabin moved after joystick movement and in what direction for all possible combinations of categories

In [ ]:
from a_data_standardization.just_stat_joy_cab_directional_ctrl import *


# PURPOSE : There was uncertainty of some experimental parameters, due to human 
# error or function of the system.

# Test different combinations of : 1) Orientation of axes (6 combinations of 
# joystick axe), yaw joystick direction was reversed, 4) Load directional control orientation,
# 5) Load deadzone margin. 

# Outputs all subject data per experimental test configuration for each experiment.

just_stat_joy_cab_directional_ctrl(varr)

## [step 2] Finding which combination of categores has the most "cabin-joystick follow"

The following cell is load_subs_into_one_matrix.py

In [ ]:
from a_data_standardization.load_subs_into_one_matrix import *


# PURPOSE : Test each experimental configuration, and determine how many trials per 
# subject followed the desired experiment rules (when the joystick was moved the
# cabin moves on the corresponding axis).

# The correct experimental configuation is the configuration that shows the most 
# counts per trial and subject for the desired experiment rules.

load_subs_into_one_matrix(varr)

Result contribution Rotation: 
-------------------------------

1) Strict = binary_marker, Lenient = direction_marker

binary_marker summed across participants, gave top correct count of categories: 'o2,y0,d0,m0,st3' (504) and 'o2,d1,m0' (485)

Thus we prove that o2 and m0 are the correct configurations given to participants, but 
the direction is ambiguous so we refer to the direction count for correct joystick cabin follow
trials.  

dir_meaning_marker summed across participants, gave top correct count category: 'o2,d0,m0' (351)

The most correct counted across participants is o2, d0, m0, denoted by the triangle.
[o2 : joystick orientation= 1)roll=17,2)pitch=16,3)yaw=18,   
d0 : directional control orientation of joystick=Convention 1 :(joy + cab = u), thus (cab = -joy), 
m0 : deadzone=0.1]

These results are in alignment with what we believed to be true, but they are confirmed.

See files 1) corcount_rot.png, 2) tdelay_rot.png, 3) dir_rot.png.

Result contribution Translation: 
-------------------------------

1) Strict = binary_marker, Lenient = direction_marker

binary_marker summed across participants, gave top correct count of categories: 'o0,d0,m0' (266) and 'o0,d1,m0' (266)

Thus we prove that o0 and m0 are the correct configurations given to participants, but 
the direction is ambiguous so we refer to the direction count for correct joystick cabin follow
trials.  

dir_meaning_marker summed across participants, gave top correct count category: 'o0,d0,m0' (187)

The most correct counted across participants is o0, d0, m0, denoted by the triangle.
[o0 : joystick orientation= 1)LR=16,2)FB=17,3)UD=18,   
d0 : directional control orientation of joystick=Convention 1 :(joy + cab = u), thus (cab = -joy), 
m0 : deadzone=0.1]

These results are in alignment with what we believed to be true, but they are confirmed.

See files 1) corcount_trans.png, 2) tdelay_trans.png, 3) dir_trans.png.

The counting analysis shows that 

1) o2,y0,d0,m0,st3 or o2,y1,d0,m0,st3 are the most counted strict-strict category combinations for rotation

2) o0,y0,d0,m0,st3 or o0,y1,d0,m0,st3 are the most counted strict-strict category combinations for rotation

The categories that are equal regardless of the orientation mean that the joystick is coupled with the cabin motion.  So, the orientation is not actually important for yaw/UD because the joystick was coupled with the cabin in the correct response.



# [B] Data preprocessing:

## [step 1] Put the script with the correct parameters: joystick orientation, directional control, deadzone. Remove the bad trials, count how many trials were removed due to specific data removal checks, and save data for next step. 

- Rotation : o2 : joystick orientation= 1)roll=17,2)pitch=16,3)yaw=18, d0 : directional control orientation of joystick=Convention 1 :(joy + cab = u), thus (cab = -joy), m0 : deadzone=0.1]
- Translation : o0 : joystick orientation= 1)LR=16,2)FB=17,3)UD=18, d0 : directional control orientation of joystick=Convention 1 :(joy + cab = u), thus (cab = -joy), m0 : deadzone=0.1]

Removal checks : 
- standardization, 
- horizontally short, 
- vertically short, 
- data cutting error (UD initialization was erroneous thought to be a trial, 
- FB or LR trials start not at zero), 
- robot stopping error (robotstall, robotjump)

In [ ]:
from b_data_preprocessing.preprocessing_pipeline import *

# PURPOSE : remove the trials that do not follow the desired experiment rules (we 
# call it standardizing the data).  Save trial cut data per subject for both
# rot and trans.  

# Creates rotdat.pkl and transdat.pkl.

plot_sub_trials = 0  # 1 = show figures, 0 = do not show figures
plotORnot_ALL = 0  # 1 = show figures, 0 = do not show figures
plot_sub_trials_FINAL = 0  # show the bad trials 
datout_temp = preprocessing_pipeline(varr, plot_sub_trials, filemarker, plotORnot_ALL, plot_sub_trials_FINAL)

## Results : axis and speed stim

We confirm that data driven axis and speed stim measures for the experiment are similar to the experimental matrix measures.

------------------------------

There were differences between the robotic/participant data and the experimental matrix parameters (axis, speed_stim).

The experimental matrix parameters were loaded from one PC, and the robot data was loaded from UDP from another PC.  If the system is in real-time there could have been delays between the PCs, and thus in writing the correct.  Also, in real-time software, the software give precedence to changing variables over fixed variables (the system was on automatic handling and/or precidence to robotic movement to prevent the system from loosing increments and stopping). So, there could be a recording delays for the fixed experimental matrix parameters in comparison to the constantly changing robot/joystick data.

------------------------------

To avoid this problem in the future one could: (1) need to use Mathscript functions to gather/control the flow of data, (2) make constraints on certain blocks to gather/control the flow of data, (3) keep orginal experimental matrix.  

By making the system less "real-time" and controlling the flow of data, data that changes fast will not get updated faster in the text document than data that changes slower.  And, the data in the text document will be more exactly aligned.  

### Speed stim correlation plots
We logically support that ssdd, data-driven measure, measuring the initial slope of cabin movemovement before the joystick was moved, is correct because the initial slope movement of the cabin is very different for sub and sup speeds, so we can decipher trial speed condition by grouping initial speed into two categories.  Due to cabin delays, the experimental matrix could have been programmed as a sup/fast trial and the cabin movement may have moved slower than needed. If this occured, the participant actually did something similar to a sub/slow trial eventhough they should have been doing a sup/fast trial; thus to try to be as close to the experimental protocol we select the data-driven measure which reflects what actually happened (the stimuli that the participants were exposed to) instead of the desired experimental matrix speed stimulus.  

The plot confirms that even for the experimental matrix speed measures, ssorg and sstas, there are discrepencies between these measures due to not being able to write to the file in a synchronised manner. For subjects 3, 4, 5, and 16 the measures are not 100 percent correlated, which they should be all at 1. It also shows that the data-driven ss measure, reflecting reality,

### Output files
1. Trial cut rotation and translation data : rotdat.pkl, transdat.pkl
2. axis correlation plots : corr_axis_out_exp_rot.png
3. speed stim correlation plots : corr_ssdir_rot.png, corr_ssmagdir_rot.png, corr_ssmag_rot.png
4. Statistics about how many trials were removed and why for each participant : trial_reject_stats_rot_1_1.png  There were 2 different criteria, and thus 4 options, for accepting trials:

a. Lenient - Lenient: (1) strictness (lenient: majority of axes had correct joystick cabin behavior), (2) strictness1 (lenient: check for joystick cabin follow behavior only)
See files : 1) trial_reject_stats_rot_0_0.png, 2) trial_reject_stats_trans_0_0.png

b. Lenient - Strict: (1) strictness (lenient: majority of axes had correct joystick cabin behavior), (2) strictness1 (strict: check for joystick cabin follow behavior and direction)
See files : 1) trial_reject_stats_rot_0_1.png, 2) trial_reject_stats_trans_0_1.png

c. Strict - Lenient: (1) strictness (strict: all axes used have correct joystick cabin behavior), (2) strictness1 (lenient: check for joystick cabin follow behavior only)
See files : 1) trial_reject_stats_rot_1_0.png, trial_reject_stats_trans_1_0.png

d. Strict - Strict: (1) strictness (strict: all axes used have correct joystick cabin behavior), (2) strictness1 (strict: check for joystick cabin follow behavior and direction)
See files : 1) trial_reject_stats_rot_1_1.png, trial_reject_stats_trans_1_1.png

## [step 2] Plot the saved data

In [69]:
def PLOTdata_from_saved_matfiles(varr, filemarker, exp, subs, plotORnot):
    
    df = pd.DataFrame()

    # ------------------------------
    # There were differences between the robotic/participant data and the experimental matrix parameters (axis, speed_stim).
    
    # The experimental matrix parameters were loaded from one PC, and the robot data was loaded from UDP from 
    # another PC.  If the system is in real-time there could have been delays between the PCs, and thus in writing
    # the correct.  Also, in real-time software, the software give precedence to changing variables over fixed
    # variables (the system was on automatic handling and/or precidence to robotic movement to prevent the system from loosing increments and stopping).  So, there could be a recording delays for the fixed experimental matrix parameters in comparison to the constantly changing robot/joystick data.
    
    # We recalculate the experimental matrix parameters (axis, speed_stim) from the robotic/participant data, called data-driven experimental parameters, to confirm the accuracy of the experimental matrix parameters
    
    # axis : the magnitude of the first 1/5 of actual cabin movement was calculated for each axis.  The axis with the largest initial magnitude is the stimulus axis.  Participants could not respond quickly, so the initial cabin movement is the true experimental stimulus.

    # speed_stim sign :  the direction of the magnitude of the first 1/5 of actual cabin movement.
    
    # Figure of experimental matrix and data-driven axis and speed_stim correlation per subject.
    # ------------------------------

    NUMmark = 0   #  an arbitrarily large number to denote a non-valid entry

    fs = 10 #  Original experimental sampling was at 250Hz, but data is downsampled to 10Hz
    ts = 1/fs

    mj_val = [0.1, 0.05, 0.01]
    
    # 1) Load exp : put the experiment that you wish to run
    Xexp = []

    if exp == 'rot':
        # Rotational data - 18 participants
        varr['which_exp'] = 'rot'
        varr['subjects'] = 'GV-123','AW-123','CDV-456','LB-000','PJ-123','PN-509','DL-777','SS-531','MD-565','CB-161','PI-112','FD-112','JMF-123','LB-195','LM-123','MBC-777','PB-123','SA-643'
        varr['sub_nom_rot'] = 'GV','AW','CDV','LB','PJ','PN','DL','SS','MD','CB','PI','FD','JMF','LB','LM','MBC','PB','SA'
        varr['anom'] = 'RO', 'PI', 'YA'
        varr['joyanom'] = '1', '2', '3'  # Numbers for standarization, because we switch axes.  For pre-processing we set an axis like in anom.
        varr['vals'] = 0.5, 1.25, 0

        varr['data_path'] = '%s%sDATA_10Hz_rot' % (varr['main_path'], filemarker)

        # 1) Orientation of joystick axes (Proven by standardization test)
        a = 17-1  # labeled (PI) - joystick movement
        b = 16-1   # labeled (RO) - joystick movement
        c = 18-1   # labeled (UD) - joystick movement

        # Load data experimental preprocessed data matrix
        file_name = "rotdat.pkl"

    elif exp == 'trans':
        # Translational data - 14 participants
        varr['which_exp'] = 'trans'
        varr['subjects'] = '20170411-TB-463','20172206-NB-777','20170413-GP-007','20172806-SM-308','20170404-AV-280','20170308-MK-160','20172008-AS-007','20170824-GL-380','p9_16102017-RW-115','p10_12102017-SG-123','p11_17102017-LG-123','p12_10102017-HS-000','p13_13102017-GB-666','p14_20171014-SL-132'
        varr['sub_nom_rot'] = 'TB','NB','GP','SM','AV','MK','AS','GL','RW','SG','LG','HS','GB','SL'        
        varr['anom'] = 'LR', 'FB', 'UD'
        varr['joyanom'] = '1', '2', '3'
        varr['vals'] = 3.75, 15, 0

        varr['data_path'] = '%s%sDATA_10Hz_trans' % (varr['main_path'], filemarker)

        # 1) Orientation of joystick axes (Proven by standardization test)
        a = 16-1  # labeled (LR) - joystick movement
        b = 17-1   # labeled (FB) - joystick movement
        c = 18-1   # labeled (UD) - joystick movement

        # Load data experimental preprocessed data matrix
        file_name = "transdat.pkl"

    # ------------------------------

    file_dir_name = "%s%s%s" % (varr['main_path2'], filemarker, file_name)
    open_file = open(file_dir_name, "rb")
    dat = pickle.load(open_file)
    open_file.close()

    corr_axis_out_exp = []
    corr_speed_stim_out_exp = []

    # ------------------------------

    # 2) Load subjects
    for s in subs:
        num_of_tr = len(dat[s][0])
        print('num_of_tr : ', num_of_tr)
        
        for tr in range(num_of_tr):
            # time series dataFrame (ensure vectors are a column): 
            # On utilise le 4 parce que c'est des donnes temporale par essaie
            num_dp = len(dat[s][4][tr][:,0])

            subject = s*np.ones((num_dp,))
            trial = tr*np.ones((num_dp,))
            
            ss_dd = dat[s][0][tr]*np.ones((num_dp,))  # 0 : speed_stim_sign : data-driven
            
            ax_dd = dat[s][1][tr]*np.ones((num_dp,))    # 1 : axis_out : data-driven
            
            new3_ind_st = dat[s][2][tr]*np.ones((num_dp,))     # 2 : new3_ind_st
            new3_ind_end = dat[s][3][tr]*np.ones((num_dp,))    # 3 : new3_ind_end
            
            outSIGCOM_ax0 = np.reshape(dat[s][4][tr][:,0], (num_dp,))  # 4 : outSIGCOM
            outSIGCOM_ax1 = np.reshape(dat[s][4][tr][:,1], (num_dp,))
            outSIGCOM_ax2 = np.reshape(dat[s][4][tr][:,2], (num_dp,))

            outSIG_ax0 = np.reshape(dat[s][5][tr][:,0], (num_dp,))    # 5 : outSIG
            outSIG_ax1 = np.reshape(dat[s][5][tr][:,1], (num_dp,))
            outSIG_ax2 = np.reshape(dat[s][5][tr][:,2], (num_dp,))

            outJOY_ax0 = np.reshape(dat[s][6][tr][:,0], (num_dp,))    # 6 : outJOY
            outJOY_ax1 = np.reshape(dat[s][6][tr][:,1], (num_dp,))
            outJOY_ax2 = np.reshape(dat[s][6][tr][:,2], (num_dp,))

            outNOISE_ax0 = np.reshape(dat[s][7][tr][:,0], (num_dp,))  # 7 : outNOISE
            outNOISE_ax1 = np.reshape(dat[s][7][tr][:,1], (num_dp,))
            outNOISE_ax2 = np.reshape(dat[s][7][tr][:,2], (num_dp,))   
            
            time = np.reshape(dat[s][8][tr], (num_dp,))        # 8 : time_org
            
            trialnum_org = dat[s][9][tr]*np.ones((num_dp,))      # 9 : trialnum_org
            
            SSQ = [dat[s][10] for i in range(num_dp)]      # 10 : SSQ - repeat the array for all entries
            
            FRT_em = dat[s][11][tr]*np.ones((num_dp,))      # 11 : FRT
            
            # ------------------------------
            
            plotthis = 0
            if plotthis == 1:
                fig, (ax0, ax1) = plt.subplots(2)
                fig.suptitle('Time data')
                ax0.plot(outSIG_ax0)
                ax0.plot(outSIG_ax1)
                ax0.plot(outSIG_ax2)
                ax0.set_ylabel('cabin')

                ax1.plot(outJOY_ax0)
                ax1.plot(outJOY_ax1)
                ax1.plot(outJOY_ax2)
                ax1.set_ylabel('joystick')

                plt.legend(loc='best')
                plt.tight_layout()
                plt.show()
            
            # ------------------------------
            
            col0 = pd.Series(subject)
            col1 = pd.Series(trial)
            col2 = pd.Series(ss_dd)
            col3 = pd.Series(ax_dd)
            col4 = pd.Series(new3_ind_st)
            col5 = pd.Series(new3_ind_end)
            col6 = pd.Series(outSIGCOM_ax0)
            col7 = pd.Series(outSIGCOM_ax1)
            col8 = pd.Series(outSIGCOM_ax2)
            col9 = pd.Series(outSIG_ax0)
            col10 = pd.Series(outSIG_ax1)
            col11 = pd.Series(outSIG_ax2)
            col12 = pd.Series(outJOY_ax0)
            col13 = pd.Series(outJOY_ax1)
            col14 = pd.Series(outJOY_ax2)
            col15 = pd.Series(outNOISE_ax0)
            col16 = pd.Series(outNOISE_ax1)
            col17 = pd.Series(outNOISE_ax2)
            col18 = pd.Series(trialnum_org)
            col19 = pd.Series(time)
            col20 = pd.Series(SSQ)
            col21 = pd.Series(FRT_em)
            
            # on peut controller l'axis par rapport quelfacon = 1
            temp = pd.concat([col0, col1, col2, col3, col4, col5, col6, col7, col8, col9,
                              col10, col11, col12, col13, col14, col15, col16, col17,
                              col18, col19, col20, col21], axis=1)
            
            df = pd.concat([df, temp], axis=0)
            # ------------------------------
    
        # ------------------------------

    col_list = ['subject', 'trial', 'ss_dd', 'ax_dd',
                'new3_ind_st', 'new3_ind_end', 'outSIGCOM_ax0', 'outSIGCOM_ax1', 
                'outSIGCOM_ax2', 'outSIG_ax0', 'outSIG_ax1', 'outSIG_ax2', 
                'outJOY_ax0', 'outJOY_ax1', 'outJOY_ax2', 'outNOISE_ax0', 
                'outNOISE_ax1', 'outNOISE_ax2', 'trialnum_org', 'time', 'SSQ',
                'FRT_em']
    df.columns = col_list
    
    # ------------------------------
    
    
            
    return df

In [ ]:
# Purpose : To plot the axis and speed correlation of experimental matrix and data-driven per subject
# and load the initial data into DataFame to confirms cut data via visualization

# We recalculate the experimental matrix parameters (axis, speed_stim) from the robotic/participant 
# data, called data-driven experimental parameters, to confirm the accuracy of the experimental 
# matrix parameters

# axis : the summed magnitude of the first 1/5 of actual cabin movement was calculated for each axis.  
# The axis with the largest initial magnitude is the stimulus axis.  Participants could not 
# respond quickly, so the initial cabin movement is the true experimental trial stimulus.

# speed_stim sign :  the direction of the summed magnitude of the first 1/5 of actual cabin movement.

# Figure of experimental matrix and data-driven axis and speed_stim correlation per subject.
# ------------------------------
plotORnot = 1
exptyp = ['rot', 'trans']
for exp in exptyp:
    if exp == 'rot':
        subs = np.arange(18)
    elif exp == 'trans':
        subs = np.arange(14)
    
    print('Plots for experiment ', exp)
    
    df = PLOTdata_from_saved_matfiles(varr, filemarker, exp, subs, plotORnot)
    df.reset_index(drop=True, inplace=True)
    
    # ------------------------------
    
    # Plot saved data to confirm that it is correct
    df.head()
    
    # ------------------------------
    
    # Cabin movement : each figure contains all trial data per SUBJECT
    sub = df.subject.value_counts().index.to_numpy()
    for s in sub:
        fig, axx = plt.subplots(nrows=1, ncols=1)
        tr = df.trial[(df.subject == s)].value_counts().index.to_numpy()
        for i in tr:
            cab0 = df.outSIG_ax0[(df.subject == s) & (df.trial == i)].to_numpy()
            cab1 = df.outSIG_ax1[(df.subject == s) & (df.trial == i)].to_numpy()
            cab2 = df.outSIG_ax2[(df.subject == s) & (df.trial == i)].to_numpy()
            plt.plot(cab0, 'r')
            plt.plot(cab1, 'g')
            plt.plot(cab2, 'b')
            axx.set_title("Subject %d : Cabin motion" % (s))
            
    # ------------------------------
    
    # Joystick movement : each figure contains all trial data per SUBJECT
    for s in sub:
        fig, axx = plt.subplots(nrows=1, ncols=1)
        tr = df.trial[(df.subject == s)].value_counts().index.to_numpy()
        for i in tr:
            joy0 = df.outJOY_ax0[(df.subject == s) & (df.trial == i)].to_numpy()
            joy1 = df.outJOY_ax1[(df.subject == s) & (df.trial == i)].to_numpy()
            joy2 = df.outJOY_ax2[(df.subject == s) & (df.trial == i)].to_numpy()
            plt.plot(joy0, 'r')
            plt.plot(joy1, 'g')
            plt.plot(joy2, 'b')
            axx.set_title("Subject %d : Joystick motion" % (s))
            
    # ------------------------------
    

# [C] Calculate metrics (parameters that describe behavior):

## Detection response type per axis:

The total possible categories of detection response are shown in the flow chart in the appendix section.

- Category 1 : Initial correct axis & direction
- Category 2 : Initial correct axis, eventually correct direction
- Category 3 : Initial correct axis, never correct direction
- Category 4 : Eventually correct axis, initially correct direction
- Category 5 : Eventually correct axis, eventually correct direction
- Category 6 : Eventually correct axis, never correct direction
- Category 7 : Never correct axis
- Category 9 : No response

*Note, category 8 and 10 are for sham/no movement trials.  We can include them, but at the moment just to keep things simple I only plot detection response for movement stimuli only.

These 10 categories can be condensed into 5 main categories that we had before:

- Category 1 : Initial Correct axis & direction
- Category 2 : Initial Correct axis, late direction
- Category 3 : Late axis & direction
- Category 4 : Never Correct
- Category 5 : No response

See the Flow chart in the paper to view the 10 logical categories of response.

## [step 1] Response type for each trial : based on decision tree/flow chart

In [ ]:
from c_calculate_metrics.detection_response import *

# ------------------------------
# detection response flow chart logic
# ------------------------------
detection_response(varr, filemarker)

# Saves the data to a file named rot_Xexp.pkl or trans_Xexp.pkl.

# Use 6th entry in the array (rot_Xexp/trans_Xexp) to get the response type across trials,
# for each participant

# [D] Plotting figures : statistical results 

1) Use rot_Xexp.pkl and trans_Xexp.pkl, and import all data into 2 pandas DataFrames (one for scalar information and one for time-series information)

2) Create Count and Time response bar charts across participants  

In [ ]:
!ipython C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Motor_classification\\
    Motor_classification\\d_main_figures_stats\\Count_TR_plot.ipynb 

# [E] SD Classification


1) Pipeline : Create X and y matricies

2) Pipeline : Classify using models

3) Plotting figures : classification results 

In [ ]:
# Pipeline
!ipython C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Motor_classification\\
    Motor_classification\\e_classification\\Creating_X_y_matricies.ipynb

In [ ]:
# Pipeline
!ipython C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Motor_classification\\
    Motor_classification\\e_classification\\SDclassification_pipeline_steps_randomsamples.ipynb

In [ ]:
# Plotting figures : Visualizations
!ipython C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Motor_classification\\Motor_classification\\e_classification\\SDclassification_Visualizations.ipynb

# [F] Correlation of perceptual disorientation with physical sickness/disorientation

In [ ]:
# Questionnaire results
!ipython C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Motor_classification\\Motor_classification\\f_SSQ_correlation\\Questionnaire_analysis.ipynb

# Extra : 

## 1) Confirmation of trial removal statistic

In [ ]:
file_name = 'C:\\Users\\jamilah\\Documents\\Github_analysis_PROJECTS\\Motor_classification\\Motor_classification\\b_data_preprocessing\\exp_sub_statistic.pkl'
open_file = open(file_name, "rb")
dataout = pickle.load(open_file)
open_file.close()

In [ ]:
len(dataout)

In [ ]:
import pandas as pd

# Let's put it in a DataFrame
exp_list = ['rot', 'trans']

df_stand_stats =  pd.DataFrame()

for exp in range(len(exp_list)):
    
    for sub in range(len(dataout[exp])):
        
        for tr in range(len(dataout[exp][sub])):
            # concat each row with the info we need
            row = pd.Series([exp, sub, dataout[exp][sub][tr], len(dataout[exp][sub])])
            row = row.rename({0: 'exp', 1: 'sub', 2: 'trial', 3: 'len'}, axis=0)
            print('row: ', row)
            
            df_stand_stats = pd.concat([df_stand_stats, row], axis=1)
            
            print('shape of df_stand_stats: ', df_stand_stats.shape)
            
            df_stand_stats.reset_index(drop=True, inplace=True)
            df_stand_stats = df_stand_stats.rename({0: 'exp', 1: 'sub', 2: 'trial', 3: 'len'}, axis=0)

df_stand_stats = df_stand_stats.T

### Way 0: 

In [ ]:
exp = 0
out = df_stand_stats[(df_stand_stats.exp == exp)]

tr_len = out['sub'].value_counts().to_numpy()
print('tr_len: ', tr_len)

tr_acc = sum(tr_len)
print('tr_acc: ', tr_acc)

print('len(tr_len): ', len(tr_len))
tr_tot = len(tr_len)*42
print('tr_tot: ', tr_tot)


num_good_tr = len(out[(out['trial'] == 0)])

# Amount of data cut 
print('num_good_tr/tr_tot', ((tr_tot-num_good_tr)/tr_tot)*100)

In [ ]:
exp = 1
out = df_stand_stats[(df_stand_stats.exp == exp)]

tr_len = out['sub'].value_counts().to_numpy()
print('tr_len: ', tr_len)

tr_acc = sum(tr_len)
print('tr_acc: ', tr_acc)

print('len(tr_len): ', len(tr_len))
tr_tot = len(tr_len)*42
print('tr_tot: ', tr_tot)


num_good_tr = len(out[(out['trial'] == 0)])

# Amount of data cut 
print('num_good_tr/tr_tot', ((tr_tot-num_good_tr)/tr_tot)*100)